<a href="https://colab.research.google.com/github/FWCJohn/Projects/blob/master/Python_ES_Options_Scalping_Bot_with_Logging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sys
import threading
import time
import logging
from datetime import datetime, timedelta

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.ticktype import TickType as tt
from ibapi.utils import iswrapper
from ibapi.common import TickerId

# --- Configuration ---
# Set up a logger to record the bot's activity.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("trading_bot.log"),
        logging.StreamHandler(sys.stdout)
    ]
)

# You must set these values to match your IBKR account and TWS/IB Gateway setup.
# Recommended to use a paper trading account and IB Gateway for stability.
IBKR_HOST = "127.0.0.1"  # Or your TWS/Gateway IP - Changed from 127.0.0.0 to 127.0.0.1
IBKR_PORT = 4002
        # Default for paper trading, 7496 for live
CLIENT_ID = 1          # A unique ID for your bot instance

# --- Strategy Parameters ---
VWAP_TICK_THRESHOLD = 500  # Number of ticks to wait before trading to get a stable VWAP
VWAP_DEVIATION = 0.05      # A percentage deviation from VWAP to trigger a trade (e.g., 0.05%)
SCALPING_TARGET = 1.5      # Price to sell at for a profit
SCALPING_STOP = -1.0       # Price to sell at for a loss

# --- Main Bot Class ---
class EOptionsScalper(EWrapper, EClient):
    """
    A scalping bot for ES options using VWAP as a directional guide.
    """

    def __init__(self):
        EClient.__init__(self, self)
        # Unique ID for our next order
        self.nextOrderId = None
        # Flag to indicate when the connection is ready to receive requests
        self.is_connected = False
        # Store for real-time market data
        self.market_data = {}
        # VWAP calculation variables
        self.vwap_data = {
            'volume': 0,
            'price_volume_sum': 0,
            'tick_count': 0
        }
        # A dictionary to manage our open positions
        self.positions = {}

    @iswrapper
    def nextValidId(self, orderId: int):
        """
        Receives the next valid order ID from IBKR.
        This is the first message received upon connection.
        """
        super().nextValidId(orderId)
        self.nextOrderId = orderId
        logging.info(f"Connected to IBKR. Next valid order ID is: {orderId}")
        self.is_connected = True

    @iswrapper
    def tickPrice(self, reqId: TickerId, tickType: int, price: float, attrib):
        """
        Receives real-time price tick data.
        """
        # Ensure we are receiving data for our subscribed ES option
        if reqId in self.market_data:
            symbol = self.market_data[reqId]['symbol']
            tick_name = tt.toStr(tickType)

            # Use 'LastPrice' (tickType 4) for our VWAP calculation and trading logic.
            # Other tick types like 'BidPrice' and 'AskPrice' could also be used.
            if tickType == 4 and price > 0:
                self.market_data[reqId]['last_price'] = price
                # We need volume to calculate VWAP
                volume = self.market_data[reqId].get('last_volume', 0)
                if volume > 0:
                    self.update_vwap(price, volume)

    @iswrapper
    def tickSize(self, reqId: TickerId, tickType: int, size: int):
        """
        Receives real-time size (volume) tick data.
        """
        if reqId in self.market_data:
            tick_name = tt.toStr(tickType)
            # Volume is reported as a separate tick type (9)
            if tickType == 9 and size > 0:
                self.market_data[reqId]['last_volume'] = size

    @iswrapper
    def contractDetails(self, reqId: int, contractDetails):
        """
        Callback to receive details about a requested contract.
        """
        logging.info(f"Received contract details for reqId {reqId}: {contractDetails.contract.localSymbol}")
        # Store the contract details so we can use them for market data and orders
        self.market_data[reqId] = {
            'symbol': contractDetails.contract.localSymbol,
            'contract': contractDetails.contract
        }

    @iswrapper
    def contractDetailsEnd(self, reqId: int):
        """
        Indicates that all contract details have been received for a request.
        """
        logging.info(f"Finished receiving contract details for reqId {reqId}")
        # Once contract details are received, we can request real-time market data
        contract_info = self.market_data.get(reqId)
        if contract_info:
            logging.info(f"Requesting market data for {contract_info['symbol']}...")
            self.reqMktData(reqId, contract_info['contract'], "", False, False, [])

    @iswrapper
    def error(self, reqId: TickerId, errorCode: int, errorString: str):
        """
        Handles error messages from the IBKR API.
        """
        if reqId > -1:
            logging.error(f"Error for request ID {reqId}: Code {errorCode} - {errorString}")
        else:
            logging.warning(f"IBKR system message: Code {errorCode} - {errorString}")

    def update_vwap(self, price, volume):
        """
        Calculates the VWAP from tick data.
        """
        self.vwap_data['price_volume_sum'] += price * volume
        self.vwap_data['volume'] += volume
        self.vwap_data['tick_count'] += 1

        # Calculate VWAP only if there is a positive volume
        if self.vwap_data['volume'] > 0:
            current_vwap = self.vwap_data['price_volume_sum'] / self.vwap_data['volume']
            self.vwap_data['vwap'] = current_vwap
            #logging.info(f"VWAP: {current_vwap:.2f} (from {self.vwap_data['tick_count']} ticks)")

    def find_es_options(self):
        """
        Finds the nearest ES options contracts.
        NOTE: This is a simplified function. A real bot would need to handle
        different expirations and strikes dynamically.
        """
        logging.info("Requesting contract details for ES futures to find options chain.")
        # Create a base ES contract for finding options
        es_contract = Contract()
        es_contract.symbol = "ES"
        es_contract.secType = "FUT"
        es_contract.exchange = "GLOBEX"
        es_contract.currency = "USD"

        # Get the current ES expiration date (e.g., today's or nearest future)
        # This part requires a more robust method to find the correct
        # futures contract and then its options. This is a placeholder.
        # A professional bot would query for all available contracts.
        es_contract.lastTradeDateOrContractMonth = "202512"  # Example expiration

        # Find all options for this futures contract
        self.reqContractDetails(1001, es_contract)

    def build_option_contract(self, symbol, right, strike, last_trade_date):
        """
        Creates an options contract object.
        """
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "FOP"  # Futures Options
        contract.exchange = "GLOBEX"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = last_trade_date
        contract.strike = strike
        contract.right = right
        return contract

    def run_strategy(self):
        """
        The main trading logic loop.
        """
        while self.is_connected:
            time.sleep(1) # Wait for a second

            # Check if we have a stable VWAP calculation
            if self.vwap_data['tick_count'] < VWAP_TICK_THRESHOLD or 'vwap' not in self.vwap_data:
                logging.info(f"Collecting market data... Current ticks: {self.vwap_data['tick_count']}/{VWAP_TICK_THRESHOLD}")
                continue

            current_vwap = self.vwap_data['vwap']

            # Find the contract to trade (this is a placeholder)
            # In a real bot, you'd find a specific strike and expiration
            contract_to_trade = next(
                (data['contract'] for data in self.market_data.values() if data.get('contract')),
                None
            )

            if not contract_to_trade:
                logging.warning("No contract to trade. Waiting for contract details.")
                continue

            last_price = self.market_data.get(contract_to_trade.conId, {}).get('last_price', None)

            if last_price is None or last_price == 0:
                continue

            logging.info(f"Current Price: {last_price:.2f} | VWAP: {current_vwap:.2f} | Pct Diff: {(last_price - current_vwap) / current_vwap * 100:.2f}%")

            # Buy signal: price is below VWAP
            if last_price < current_vwap * (1 - VWAP_DEVIATION):
                if not self.positions:
                    logging.info(f"Price ({last_price:.2f}) is significantly below VWAP ({current_vwap:.2f}). BUY SIGNAL.")
                    # A scalper would typically buy calls or sell puts.
                    # This example buys a call.
                    self.place_order(contract_to_trade, "BUY", 1)

            # Sell signal: price is above VWAP
            elif last_price > current_vwap * (1 + VWAP_DEVIATION):
                if not self.positions:
                    logging.info(f"Price ({last_price:.2f}) is significantly above VWAP ({current_vwap:.2f}). SELL SIGNAL.")
                    # A scalper would typically buy puts or sell calls.
                    # This example sells a call.
                    self.place_order(contract_to_trade, "SELL", 1)

            # Check for exiting positions
            for orderId, position in list(self.positions.items()):
                entry_price = position['entry_price']
                current_profit = last_price - entry_price

                # Check for take-profit or stop-loss
                if current_profit >= SCALPING_TARGET or current_profit <= SCALPING_STOP:
                    logging.info(f"Exiting position {orderId}. Current profit/loss: {current_profit:.2f}")
                    self.place_order(contract_to_trade, "SELL", 1, is_exit=True)

    def place_order(self, contract, action, quantity, is_exit=False):
        """
        Places an order with a unique order ID.
        """
        # Ensure we have a valid order ID
        if self.nextOrderId is None:
            logging.error("Cannot place order. No valid order ID received.")
            return

        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "MKT" # A simple market order for this example

        if is_exit:
            order.transmit = True
            # For simplicity, we assume we only have one position to close
            self.positions = {}
            logging.info(f"Market order to {action} {quantity} of {contract.symbol} sent (Exit). OrderId: {self.nextOrderId}")
        else:
            # We assume a single position for simplicity in this example
            self.positions[self.nextOrderId] = {
                'entry_price': self.market_data[contract.conId]['last_price'],
                'contract': contract
            }
            order.transmit = True
            logging.info(f"Market order to {action} {quantity} of {contract.symbol} sent. OrderId: {self.nextOrderId}")

        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1

    def run(self):
        """
        Main function to start the bot.
        """
        # Start the connection loop in a separate thread
        thread = threading.Thread(target=self.run_strategy)
        thread.start()

        # Start the IBKR client connection
        try:
            logging.info(f"Attempting to connect to IBKR at {IBKR_HOST}:{IBKR_PORT} with client ID {CLIENT_ID}...")
            self.connect(IBKR_HOST, IBKR_PORT, CLIENT_ID)

            # Wait for a connection and then request contract details
            start_time = time.time()
            # Increased timeout to 30 seconds
            while not self.is_connected and time.time() - start_time < 30:
                time.sleep(0.5)

            if not self.is_connected:
                 logging.critical("Failed to connect to IBKR within timeout. Please ensure TWS/Gateway is running and API settings are correct.")
                 self.disconnect()
                 return

            # Request contract details for the newest ES options contract
            # This will trigger the contractDetails callback and then market data request
            self.find_es_options()

            # Start the client's message processing loop
            self.run()

        except Exception as e:
            logging.critical(f"An error occurred during connection or execution: {e}")
            self.disconnect()


def main():
    bot = EOptionsScalper()
    bot.run()

if __name__ == "__main__":
    main()

is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002
CRITICAL:root:Failed to connect to IBKR within timeout. Please ensure TWS/Gateway is running and API settings are correct.


In [2]:
!pip install ibapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ibapi: filename=ibapi-9.81.1.post1-py3-none-any.whl size=67942 sha256=13a4cf967f12cffda87d8ceab7c8e449ad00318be758239ce160769a754550e1
  Stored in directory: /root/.cache/pip/wheels/07/e7/d4/b94d0968474f56f8a2c3eafe3debcc340f15ef1ba362eb38a8
Successfully built ibapi
